In [74]:
import pandas as pd
import numpy as np
import MICE
import iMICE

df_ds = pd.read_csv('Admission_Predict_Ver1.1.csv')
df=df_ds.copy()
df['Chance']=df.iloc[:,8].values.copy()

df_1=df.copy()
df_1 = df_1.drop(df_1.columns[[9]],axis = 1)
df_1['ChanceBinary'] = df['Chance'].apply(lambda x: 1 if x >= 0.70 else 0)

X = df_1.iloc[:, [1,2,6]].values  
y = df_1.iloc[:, 9].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

#y_test
df_2n = pd.DataFrame({'Chance':y_test[:]})
df_2n.at[:,'Chance']=np.nan
y_test_n=df_2n.iloc[:,0].values

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'ChanceOfAdmit':y_train_1[:],'GREScore':X_train_1[:,0],'TOEFLScore':X_train_1[:,1],'CGPA':X_train_1[:,2]})


# MICE

In [75]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3 = pd.DataFrame(mice_result, columns = ['ChanceOfAdmit', 'GREScore', 'TOEFLScore', 'CGPA'])
predict = df_3['ChanceOfAdmit']

predict = np.array(predict)

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print('Accuracy:',accuracy_score(y_test, predict[r1:]))
print(f1_score(y_test, predict[r1:]))
print(precision_score(y_test, predict[r1:]))
print(recall_score(y_test, predict[r1:]))

#print(classification_report(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predict[r1:])) 

Accuracy: 0.75
0.782608695652174
0.7258064516129032
0.8490566037735849
[[30 17]
 [ 8 45]]


# iMICE

In [76]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['ChanceOfAdmit', 'GREScore', 'TOEFLScore', 'CGPA'])
predicti = df_3i['ChanceOfAdmit'].map(lambda x: int(x))

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print('Accuracy:',accuracy_score(y_test, predicti[r1:]))
print(f1_score(y_test, predicti[r1:]))
print(precision_score(y_test, predicti[r1:]))
print(recall_score(y_test, predicti[r1:]))
print(confusion_matrix(actual[r1:], predicti[r1:]))

Accuracy: 0.83
0.8495575221238938
0.8
0.9056603773584906
[[35 12]
 [ 5 48]]


# KNN

In [77]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.84
[[38  9]
 [ 7 46]]


# Random Forest

In [78]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.83
[[37 10]
 [ 7 46]]
